In [1]:
import os

import numpy as np
import pandas as pd

import cloudpickle

from transformers import *
import torch

2021-10-22 21:53:52.787786: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-22 21:53:52.787903: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
matchings = pd.read_csv('../input/combiner-target-dataset-0-of-5/final_matchings.csv')

matchings.head(3)

,image_url,page_title,caption_reference_description,count,spaced_filename,spaced_filename_translit,title_translit,caption_translit,page_title_lang,page_title_lang_p,...,caption_lang,caption_lang_p,caption_en,caption_contains_digit,undigit_caption,page_title_contains_digit,undigit_page_title,target,caption_translation,final_caption
0,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens's Justitia, 1663-1665, Museumsber...",4,Jurgen ovens justitia,NaN,Juergen Ovens,"Juergen Ovens's Justitia, 1663-1665, Museumsbe...",en,0.124504,...,en,0.894902,True,True,"Jürgen Ovens's Justitia, -, Museumsberg Flensb...",False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens's Justitia, 16...",NaN,"Jürgen Ovens's Justitia, 1663-1665, Museumsber..."
1,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens: Justitia, \n 1663-1665, Museumsb...",4,Jurgen ovens justitia,NaN,Juergen Ovens,"Juergen Ovens: Justitia, \n 1663-1665, Museums...",en,0.124504,...,en,0.588354,True,True,"Jürgen Ovens: Justitia, \n -, Museumsberg, Fle...",False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens: Justitia, \n ...",NaN,"Jürgen Ovens: Justitia, \n 1663-1665, Museumsb..."
2,https://upload.wikimedia.org/wikipedia/commons...,"Овенс, Юрген","Юрген Овенс Юстиция, 1663—1665, Музей Фленсбур...",4,Jurgen ovens justitia,NaN,"Ovens, Jurgen","Jurgen Ovens Justitsija, 1663—1665, Muzej Flen...",ru,0.647271,...,ru,0.949836,False,True,"Юрген Овенс Юстиция, —, Музей Фленсбург. Овенс...",False,"Овенс, Юрген","Овенс, Юрген [SEP] Юрген Овенс Юстиция, 1663—1...","Jurgen Owens Justice, -, Flensburg Museum. Owe...","Jurgen Owens Justice, -, Flensburg Museum. Owe..."


In [3]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
def retrieve_entities(pairs, verbose=False):
    def repair(s):
        return s.replace(' #', '').replace('#', '')
    def unique(l):
        return list(set(l))
    
    
    reading_entity = False
    current_entity_type = None
    
    misc, per, org, loc = [], [], [], []
    
    entity_parts = []
    
    for part, et in pairs:
        if not reading_entity and et == 'O':
            continue
        else:
            if not reading_entity:
                current_et = et
                reading_entity = True               
                
            if current_et == et:
                entity_parts.append(part)
            else:
                repaired_entity = repair(' '.join(entity_parts))
                
                if verbose:
                    print(repaired_entity)
                    
                locals()[current_et.split('-')[-1].lower()].append(repaired_entity)
                entity_parts.clear()
                
                if et == 'O':
                    reading_entity = False
                    current_et = None
                else:
                    entity_parts.append(part)
                    current_et = 'I-' + et.split('-')[-1]
                    
                    
    return { 'misc': unique(misc), 'per': unique(per), 'org': unique(org), 'loc': unique(loc)}

In [5]:
image_entities = { }

for index, row in matchings.loc[0:100000].iterrows():
    try:
        sequence = row['final_caption'].strip()

        if sequence != '':
            tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
            inputs = tokenizer.encode(sequence, return_tensors="pt")

            outputs = model(inputs)[0]
            predictions = torch.argmax(outputs, dim=2)

            l = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]

            image_entities[index] = retrieve_entities(l)
    except Exception as ex:
        print(index, sequence)
        print(str(ex))
        print()

2658 The stone foundation of the main tower of Edo Castle.
'float' object has no attribute 'strip'

16261 Freida Pinto (2010)
'float' object has no attribute 'strip'

18814 Marc Madiot, pictured here at the start of the th stage of the Four Days from Dunkirk to Barlin, runs for the team from to.
'float' object has no attribute 'strip'

33585 Manning drops back to pass on January 15, 2012.
'float' object has no attribute 'strip'

34861 Malta 1980
'float' object has no attribute 'strip'

54668 Ferdinand Lassalle
'float' object has no attribute 'strip'

61699 The Vinodol law is the most important and the oldest transcript originating from. century, consists of sheets (or pages) of a parchment notebook (x mm) in italic Glagolitic script is written page (several introductory lines in literary Glagolitic script), and the other pages are blank. The manuscript contains several initials, drawings and later attributions. It is kept today in the National and University Library in Zagreb.
'float' 

In [6]:
with open('image_entities.dump', 'wb') as file:
    cloudpickle.dump(image_entities, file)